In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from PIL import ImageFile

# Allow loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_INITIAL = 20
EPOCHS_FINE_TUNE = 10
DATA_DIR = "Mushrooms"  # Training and validation dataset directory

# Step 1: Data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Train & Validation Generators
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

# Step 2: Load MobileNetV2 base
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base initially

# Step 3: Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Step 4: Compile the model
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 5: Initial training
print("Starting initial training...")
history_initial = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_INITIAL,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

# Step 6: Accuracy after initial training
initial_val_acc = history_initial.history['val_accuracy'][-1]
print(f"\nModel Accuracy before fine-tuning: {initial_val_acc:.4f}")

# Step 7: Fine-tuning
print("\nUnfreezing and starting fine-tuning...")
base_model.trainable = True
model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

# Step 8: Final evaluation on validation
final_val_acc = history_fine.history['val_accuracy'][-1]
final_train_acc = history_fine.history['accuracy'][-1]
print(f"\nFinal Validation Accuracy: {final_val_acc * 100:.2f}%")
print(f"Final Training Accuracy: {final_train_acc * 100:.2f}%")

# Step 9: Save model and weights
model.save("mushroom_model.h5")
model.save_weights("mushroom_model_weights.h5")
print(" Final Model Saved as 'mushroom_model.h5'")
print(" Final Weights Saved as 'mushroom_model_weights.h5'")


Found 5375 images belonging to 9 classes.
Found 1339 images belonging to 9 classes.


Starting initial training...
Epoch 1/20
168/168 [==============================] - 56s 328ms/step - loss: 1.2475 - accuracy: 0.5736 - val_loss: 1.1191 - val_accuracy: 0.6094
Epoch 2/20
168/168 [==============================] - 58s 342ms/step - loss: 0.8798 - accuracy: 0.6945 - val_loss: 1.0252 - val_accuracy: 0.6453
Epoch 3/20
168/168 [==============================] - 69s 412ms/step - loss: 0.7652 - accuracy: 0.7336 - val_loss: 1.0360 - val_accuracy: 0.6400
Epoch 4/20
168/168 [==============================] - 81s 480ms/step - loss: 0.7042 - accuracy: 0.7498 - val_loss: 1.1035 - val_accuracy: 0.6206



Model Accuracy before fine-tuning: 0.6206

Unfreezing and starting fine-tuning...
Epoch 1/10
168/168 [==============================] - 280s 2s/step - loss: 1.1094 - accuracy: 0.6128 - val_loss: 1.0086 - val_accuracy: 0.6460
Epoch 2/10
168/168 [==============================] - 278s 2s/step - loss: 0.8787 - accuracy: 0.7021 - val_loss: 1.0042 - val_accuracy: 0.6580
Epoch 3/10
168/168 [==============================] - 277s 2s/step - loss: 0.7910 - accuracy: 0.7233 - val_loss: 1.0168 - val_accuracy: 0.6490
Epoch 4/10
168/168 [==============================] - 281s 2s/step - loss: 0.7110 - accuracy: 0.7596 - val_loss: 0.9971 - val_accuracy: 0.6624
Epoch 5/10
168/168 [==============================] - 321s 2s/step - loss: 0.6596 - accuracy: 0.7682 - val_loss: 0.9814 - val_accuracy: 0.6535
Epoch 6/10
168/168 [==============================] - 359s 2s/step - loss: 0.6010 - accuracy: 0.7927 - val_loss: 0.9874 - val_accuracy: 0.6692
Epoch 7/10
168/168 [==============================] - 322s 